In [1]:
import sys
import itertools
from pprint import pprint
from copy import deepcopy
import numpy as np

In [2]:
def my_updateClusters(ll_concatenate, beta):
    
    def AssignPointsToClusters():
    
        AllComb = [list(comb) for comb in itertools.product([k for k in range(K)], repeat = r_w)]
        #print(AllComb)

        PrevCost = [0] * len(AllComb)
        CurrCost = [float('Inf')] * len(AllComb)
        PrevPath = [ [] for _ in range(len(AllComb)) ]
        CurrPath = [ [] for _ in range(len(AllComb)) ]
        #print("PrevCost: " + str(PrevCost))
        #print("CurrCost: " + str(CurrCost))
        #print("PrevPath: " + str(PrevPath))
        #print("CurrPath: " + str(CurrPath))

        for t in range(T):
            CurrCost = [float('Inf')] * len(AllComb)
            for CurrIndex in range(len(AllComb)):
                for PrevIndex in range(len(AllComb)):
                    CurrCostPlusPrevCost = CalCost(PrevCost[PrevIndex], PrevIndex, CurrIndex, t, ll[t], beta, alpha, AllComb)
                    if CurrCostPlusPrevCost < CurrCost[CurrIndex]:
                        CurrCost[CurrIndex] = CurrCostPlusPrevCost
                        CurrPath[CurrIndex] = deepcopy(PrevPath[PrevIndex])
                        CurrPath[CurrIndex].append(AllComb[CurrIndex])
            PrevCost = deepcopy(CurrCost)
            PrevPath = deepcopy(CurrPath)

        for Index in range(len(AllComb)):
            print("CurrPath: " + str(CurrPath[Index]) + " (CurrCost: " + str(CurrCost[Index]) + ")")

        FinalMinIndex = CurrCost.index(min(CurrCost))
        FinalPath = CurrPath[FinalMinIndex]
        FinalCost = CurrCost[FinalMinIndex]
        print("FinalPath: " + str(FinalPath))
        print("FinalCost: " + str(CurrCost[FinalMinIndex]))

        return FinalPath, FinalCost
    
    def CalCost(PrevCost, PrevIndex, CurrIndex, t, ll, beta, alpha, AllComb):
    
        CurrCost = PrevCost

        for r in range(r_w):
            CurrCost += ll[r][ AllComb[CurrIndex][r] ]
            #print(ll[r][ AllComb[CurrIndex][r] ])

        # consider LTC
        if t != 0:
            for r in range(r_w):
                if AllComb[PrevIndex][r] != AllComb[CurrIndex][r]:
                    CurrCost += beta
                    #print("PLUS beta")

        # consider GTC
        for r in range(r_w):
            if r != 0 and AllComb[CurrIndex][r-1] != AllComb[CurrIndex][r]:
                CurrCost += alpha
                #print("PLUS alpha")

        return CurrCost
    
    
    
    ll_concatenate = ll_concatenate.tolist()
    
    # change ll_concatenate to ll
    ll = [ [] for _ in range(T) ]
    
    for t in range(T):
        for ll_concatenate_idx in range(T * r_w):
            if ll_concatenate_idx % T == t:
                ll[t].append(ll_concatenate[ll_concatenate_idx])
    
    
    # call AssignPointsToClusters
    FinalPath, FinalCost = AssignPointsToClusters()
    
    
    # change FinalPath to FinalPath_concatenate
    FinalPath_concatenate = []

    for r in range(r_w):
        for t in range(T):
            FinalPath_concatenate.append(FinalPath[t][r])
    
    return np.asarray(FinalPath_concatenate)

In [3]:
def updateClusters(LLE_node_vals, beta):
        """
        Takes in LLE_node_vals matrix and computes the path that minimizes
        the total cost over the path
        Note the LLE's are negative of the true LLE's actually!!!!!

        Note: switch penalty > 0
        """
        (T, num_clusters) = LLE_node_vals.shape
        future_cost_vals = np.zeros(LLE_node_vals.shape)

        # compute future costs
        for i in range(T-2, -1, -1):
            j = i+1
            indicator = np.zeros(num_clusters)
            future_costs = future_cost_vals[j, :]
            lle_vals = LLE_node_vals[j, :]
            for cluster in range(num_clusters):
                total_vals = future_costs + lle_vals + beta
                total_vals[cluster] -= beta
                future_cost_vals[i, cluster] = np.min(total_vals)

        # compute the best path
        path = np.zeros(T)

        # the first location
        curr_location = np.argmin(future_cost_vals[0, :] + LLE_node_vals[0, :])
        path[0] = curr_location

        # compute the path
        for i in range(T-1):
            j = i+1
            future_costs = future_cost_vals[j, :]
            lle_vals = LLE_node_vals[j, :]
            total_vals = future_costs + lle_vals + beta
            total_vals[int(path[i])] -= beta

            path[i+1] = np.argmin(total_vals)
        print(total_vals)
        # return the computed path
        return path

In [4]:
beta = 4
alpha = 0
T = 3
K = 4
r_w = 3

In [5]:
ll_concatenate = [ [] for _ in range(T * r_w) ]

t = 0
ll_concatenate[t].append(3)
ll_concatenate[t].append(1)
ll_concatenate[t].append(3)
ll_concatenate[t].append(5)
t += 1
ll_concatenate[t].append(0)
ll_concatenate[t].append(7)
ll_concatenate[t].append(2)
ll_concatenate[t].append(4)
t += 1
ll_concatenate[t].append(17)
ll_concatenate[t].append(0)
ll_concatenate[t].append(31)
ll_concatenate[t].append(40)
t += 1
ll_concatenate[t].append(2)
ll_concatenate[t].append(4)
ll_concatenate[t].append(3)
ll_concatenate[t].append(1)
t += 1
ll_concatenate[t].append(5)
ll_concatenate[t].append(4)
ll_concatenate[t].append(3)
ll_concatenate[t].append(1)
t += 1
ll_concatenate[t].append(10)
ll_concatenate[t].append(22)
ll_concatenate[t].append(25)
ll_concatenate[t].append(9)
t += 1
ll_concatenate[t].append(7)
ll_concatenate[t].append(5)
ll_concatenate[t].append(3)
ll_concatenate[t].append(4)
t += 1
ll_concatenate[t].append(4)
ll_concatenate[t].append(6)
ll_concatenate[t].append(6)
ll_concatenate[t].append(4)
t += 1
ll_concatenate[t].append(20)
ll_concatenate[t].append(19)
ll_concatenate[t].append(1)
ll_concatenate[t].append(23)

ll_concatenate = np.asarray(ll_concatenate)

print("ll_concatenate: " + str(ll_concatenate))

ll_concatenate: [[ 3  1  3  5]
 [ 0  7  2  4]
 [17  0 31 40]
 [ 2  4  3  1]
 [ 5  4  3  1]
 [10 22 25  9]
 [ 7  5  3  4]
 [ 4  6  6  4]
 [20 19  1 23]]


In [6]:
FinalPath_concatenate = my_updateClusters(ll_concatenate, beta)
print(FinalPath_concatenate)

CurrPath: [[0, 3, 0], [0, 3, 0], [0, 0, 0]] (CurrCost: 67)
CurrPath: [[0, 3, 1], [0, 3, 1], [0, 0, 1]] (CurrCost: 66)
CurrPath: [[0, 3, 2], [0, 3, 2], [0, 0, 2]] (CurrCost: 46)
CurrPath: [[0, 3, 3], [0, 3, 3], [0, 0, 3]] (CurrCost: 67)
CurrPath: [[0, 3, 0], [0, 3, 0], [0, 1, 0]] (CurrCost: 79)
CurrPath: [[0, 3, 1], [0, 3, 1], [0, 1, 1]] (CurrCost: 78)
CurrPath: [[0, 3, 2], [0, 3, 2], [0, 1, 2]] (CurrCost: 58)
CurrPath: [[0, 3, 3], [0, 3, 3], [0, 1, 3]] (CurrCost: 79)
CurrPath: [[0, 2, 0], [0, 2, 0], [0, 2, 0]] (CurrCost: 82)
CurrPath: [[0, 2, 1], [0, 2, 1], [0, 2, 1]] (CurrCost: 81)
CurrPath: [[0, 2, 2], [0, 2, 2], [0, 2, 2]] (CurrCost: 61)
CurrPath: [[0, 2, 3], [0, 2, 3], [0, 2, 3]] (CurrCost: 82)
CurrPath: [[0, 3, 0], [0, 3, 0], [0, 3, 0]] (CurrCost: 62)
CurrPath: [[0, 3, 1], [0, 3, 1], [0, 3, 1]] (CurrCost: 61)
CurrPath: [[0, 3, 2], [0, 3, 2], [0, 3, 2]] (CurrCost: 41)
CurrPath: [[0, 3, 3], [0, 3, 3], [0, 3, 3]] (CurrCost: 62)
CurrPath: [[0, 3, 0], [0, 3, 0], [1, 0, 0]] (CurrCost: 5

In [7]:
FinalPath_concatenate = updateClusters(ll_concatenate, beta)
print(FinalPath_concatenate)

[ 24.  23.   5.  23.]
[ 0.  0.  1.  3.  3.  3.  3.  3.  2.]
